# SISTEMA ZERO-SHOT
Este sistema detecta TTPs de desinformación de DISARM en una noticia de entrada sin tener en cuenta contexto relevante..

Este código carga el modelo de Mistral 7B Instruct y las TTPs que se pueden detectar, y ejecuta el LLM con una noticia para generar una respuesta con los patrones identificados y una justificación.

Se deben haber instalado las dependencias necesarias (requirements.txt) y disponer de un token de HuggingFace.

In [1]:
import json

# Cargar las TTPs de DISARM que se pueden detectar
def cargar_ttps(path="DISARMfiltrado.json"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    bloques = []
    for obj in data["objects"]:
        if obj["type"] == "tactic":
            tactic_name = obj["name"]
            for tech in obj.get("techniques", []):
                id_ = tech["id"]
                name = tech["name"]
                desc = tech["description"].strip()
                bloques.append(f"Technique: {id_} - {name} (Tactic: {tactic_name})\nTechnique Description: {desc}")
    return "\n\n".join(bloques)

In [1]:
from huggingface_hub import login

# Iniciar sesión en HuggingFace para acceder al modelo
login(token="")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Cargar el modelo Mistral
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [3]:
# Función para construuir el prompt y generar la respuesta
def analyse(news_text):
    ttps=cargar_ttps()
    prompt =  f"""
        You are an expert in communication, disinformation, and discourse analysis. You will now receive one news article to analyze and a list of known manipulation and disinformation techniques (DISARM TTPs).
                
        Your task is to determine whether the ARTICLE contains any patterns of manipulation or disinformation.
        
        Use the TTPs section below as the reference list of patterns to detect.
                
        Return the result **strictly in valid JSON format** with this structure:
        
        {{
            "ttps_detected": [string], //TTP names, example: ["T0015-Create Hashtags and Search Artefacts"]
            "key_phrases": [string], //Phrases or evidence from the article that support your findings
            "justification": string //A concise explanation of your reasoning. Every TTP mentioned in the justification must be listed in the ttps_detected array.
        }}
                
        If no clear technique is detected, respond:
                
        {{
            "ttps_detected": [],
            "key_phrases": [],
            "justification": "No clear patterns of manipulation have been detected."
        }}
        
                
        ARTICLE:
        {news_text}
        
        TTPs (DISARM framework):
        {ttps_text}
        
        RESPONSE:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    # Quitar el prompt para quedarnos con la respuesta
    return result[len(prompt):].strip()

In [4]:
# NOTICIA QUE SE QUIERE ANALIZAR
noticia = "BUSTED: Leaked Documents Prove Trump Took Laundered Money From Russian Bank. Convincing President Trump to release his tax returns is proving slightly more difficult than we initially anticipated, but that doesn t mean there haven t been any signs of success from taking the longer route. Take, for example, a 98-page document recently released by the United States Office of Government Ethics.The document, available in its entirety here, clearly shows that not only is Donald Trump outright profiting from the presidency, a direct violation of the Emoluments Clause of the United States Constitution but also that he is in debt to several banks, both domestic and foreign.Although shocking news, none of this particularly comes as a surprise, more or less just confirms what most of us already suspected. However, it s when you delve into the details that you discover the true significance of the President s possible under-the-table actions.German-based Deutsche Bank was served with $630 million in penalties back in January for a $10 billion Russian money laundering scheme involving it s Moscow and New York branches among others. Deutsche Bank also gave Trump four questionable long-term  loans,  potentially Russian money that s been handed to Trump, the final loan given just before the commencement of the presidential election and used to help fund the Trump International Hotel that opened in Washington DC last year.Obviously, this interaction between Trump and Deutsche Bank should raise more than just a few red flags, at least confirming that there is a possibility that the Russians laundered money to Trump as he began his campaign and just prior to hiring senior advisors with ties to the Russian government.Furthermore, Deutsche Bank is gaining somewhat of a reputation for their shady business dealings as well. Not only were they caught in the Russian laundering scheme, the bank also struck a $7.2 billion deal with the US government last December to settle claims for toxic mortgages they packaged and sold between 2005 and 2007, as well as paying $2.5 billion in April 2015 to settle charges it conspired to manipulate global interest rate benchmarks.It s interesting, yet not at all surprising, how corrupt people and organizations just seem to gravitate toward each other."

# RESPUESTA DEL LLM
print(analyse(noticia))

{
        "ttps_detected": ["T0075.001 - Discredit Credible Sources", "T0022.001 - Amplify Existing Conspiracy Theory Narratives"],
        "key_phrases": ["Leaked Documents Prove Trump Took Laundered Money From Russian Bank", "Deutsche Bank also gave Trump four questionable long-term loans", "Obviously, this interaction between Trump and Deutsche Bank should raise more than just a few red flags"],
        "justification": "The article presents leaked documents as evidence of Trump taking laundered money from a Russian bank, which is a conspiracy theory not supported by mainstream sources. The article also amplifies this conspiracy theory by stating that Deutsche Bank gave Trump four questionable long-term loans, which is a claim that has been debunked by fact-checkers. The article's phrasing, such as 'Obviously, this interaction between Trump and Deutsche Bank should raise more than just a few red flags', suggests an attempt to discredit credible sources by implying that questioning t